In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import time
import re

In [ ]:
from sklearn.svm import SVC

In [ ]:
import sklearn.svm

In [ ]:
import xgboost as xgb

In [ ]:
! pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 10.7 MB/s eta 0:00:00


In [ ]:
!pip install optuna-integration

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 3.2 MB/s eta 0:00:00


In [ ]:
import optuna

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [ ]:
filename = "/content/drive/My Drive/Datasets/s_2k.csv"

df = pd.read_csv(filename,
                 encoding="utf-8", encoding_errors="replace")
# replace values
df.replace({'Not Social': 0, 'Social': 1}, inplace=True)

# Data Preprocessing

In this section we will prepare the dataset in order to use in SVM model. Before starting I want to explain processes with 1 line explanations



*   Cleaning Everything Except Alphabetical and Numerical Characters: We'll clean unrelevant digits using regular expressions
*   Tokenizing and Lemmatizing: We'll split texts into their words and convert words to their base form (dogs=>dog)
*   Removing Stopwords: We'll remove words that have no special meaning (such as the,will,was)
*   Joining Prepared Strings: We'll join the words
*   Bag of Words Approach: We'll create a bag of word. In bag of words approach each feature shows whether the text contains the word or not.







In [ ]:
cleanedData = []

lemma = WordNetLemmatizer()
swords = stopwords.words("english")
for text in df["text"]:

    # Cleaning everything except alphabetical and numerical characters
    text = re.sub("[^a-zA-Z0-9]"," ",text)

    # Tokenizing and lemmatizing
    text = nltk.word_tokenize(text.lower())
    text = [lemma.lemmatize(word) for word in text]

    # Removing stopwords
    text = [word for word in text if word not in swords]

    # Joining
    text = " ".join(text)

    cleanedData.append(text)

In [ ]:
# check if the data is cleaned
for i in range(0,5):
    print(cleanedData[i],end="\n\n")

balance included kind donation prepared accordance b4si guideline

internal control risk management board audit committee responsible ensuring suitable internal control system prevent detect fraud error designed implemented third party service provider company also responsible reviewing effectiveness control

analysis doe take account energy mix related electricity consumption ttv portfolio issuer

change chief financial officer plc board chair position clare bousfield stepped board effect 1 october 2021 transitioned role managing director retail saving business clare continued receive salary benefit reflective new role

consideration prioritisation climate risk also built decision making governance process requirement key strategic board risk assessment paper



Now we can create our bag of words

In [ ]:
vectorizer = CountVectorizer(max_features=10000)
BOW = vectorizer.fit_transform(cleanedData)

we can split our set into train and test.

In [ ]:
x_train = list()
x_test = list()
y_train = list()
y_test = list()
for label in [0, 1]:
  x_train,x_test,y_train,y_test = train_test_split(BOW, np.asarray(df["label"]),
                                                  train_size=250,
                                                  test_size=250,
                                                  random_state=42)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(250, 4596)
(250, 4596)
(250,)
(250,)


Support Vector Machine Classifier Modeling

In [ ]:
start_time = time.time()

model = SVC()
model.fit(x_train,y_train)

end_time = time.time()
process_time = round(end_time-start_time,2)
print("Fitting SVC took {} seconds".format(process_time))

Fitting SVC took 0.02 seconds


In [ ]:
predictions = model.predict(x_test)

In [ ]:
print("Accuracy of model is {}%".format(accuracy_score(y_test,predictions) * 100))

Accuracy of model is 72.39999999999999%


# Hyperparameter tuning using Optuna

In [ ]:
clf = SVC(gamma="auto")
param_distributions = {
        "C": optuna.distributions.FloatDistribution(1e-10, 1e10, log=True),
        "degree": optuna.distributions.IntDistribution(1, 5),
    }

optuna_search = optuna.integration.OptunaSearchCV(
        clf, param_distributions, n_trials=100, timeout=600, verbose=2
    )


optuna_search.fit(x_train,y_train)

print("Best trial:")
trial = optuna_search.study_.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

<ipython-input-22-4062d26f2c32>:7: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(
[I 2024-04-30 03:04:30,401] A new study created in memory with name: no-name-b5c85214-ebc5-4197-bb32-8f0cafbd30d2
INFO:optuna_integration.sklearn:Searching the best hyperparameters using 250 samples...
[I 2024-04-30 03:04:30,472] Trial 0 finished with value: 0.752 and parameters: {'C': 3947007136.0451965, 'degree': 5}. Best is trial 0 with value: 0.752.
[I 2024-04-30 03:04:30,525] Trial 1 finished with value: 0.62 and parameters: {'C': 2.07183680508273e-05, 'degree': 3}. Best is trial 0 with value: 0.752.
[I 2024-04-30 03:04:30,586] Trial 2 finished with value: 0.62 and parameters: {'C': 19.03021573814445, 'degree': 1}. Best is trial 0 with value: 0.752.
[I 2024-04-30 03:04:30,644] Trial 3 finished with value: 0.62 and parameters: {'C': 0.00012035364110391863, 'degree': 1}. Best is t

Best trial:
  Value:  0.788
  Params: 
    C: 237.12419168900635
    degree: 5


In [ ]:
trial.params

{'C': 237.12419168900635, 'degree': 5}

In [ ]:
start_time = time.time()

model = SVC(gamma="auto", degree=trial.params['degree'], C=trial.params['C'])
model.fit(x_train,y_train)

end_time = time.time()
process_time = round(end_time-start_time,2)
print("Fitting SVC took {} seconds".format(process_time))

Fitting SVC took 0.02 seconds


In [ ]:
trial.params['degree']

5

In [ ]:
predictions = model.predict(x_test)

In [ ]:
print("Accuracy of model is {}%".format(accuracy_score(y_test,predictions) * 100))

Accuracy of model is 82.0%


In [ ]:
def evaluate(y_true, y_pred):
    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true))
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1])
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [ ]:
evaluate(y_true=y_test, y_pred=predictions)

Accuracy: 0.820
Accuracy for label 0: 0.855
Accuracy for label 1: 0.765

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.86      0.85       152
           1       0.77      0.77      0.77        98

    accuracy                           0.82       250
   macro avg       0.81      0.81      0.81       250
weighted avg       0.82      0.82      0.82       250


Confusion Matrix:
[[130  22]
 [ 23  75]]


# build model for XGBoost

In [ ]:
start_time = time.time()

xgb_model = xgb.XGBClassifier()
xgb_model.fit(x_train,y_train)

end_time = time.time()
process_time = round(end_time-start_time,2)
print("Fitting XGBoost took {} seconds".format(process_time))

Fitting XGBoost took 1.64 seconds


In [ ]:
predictions = model.predict(x_test)

In [ ]:
print("Accuracy of model is {}%".format(accuracy_score(y_test,predictions) * 100))

Accuracy of model is 82.0%


# Hyperparameter tuning using Optuna

In [ ]:
def objective(trial):
    dtrain = xgb.DMatrix(x_train, label=y_train)
    dvalid = xgb.DMatrix(x_test, label=y_test)

    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")
    bst = xgb.train(param, dtrain, evals=[(dvalid, "validation")], callbacks=[pruning_callback])
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    accuracy = accuracy_score(y_test,predictions)
    return accuracy

In [ ]:
study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), direction="maximize"
    )
study.optimize(objective, n_trials=100)
print(study.best_trial)

[I 2024-04-30 03:04:39,592] A new study created in memory with name: no-name-befdab8d-9ce5-4a64-b799-139cdb42533a


[0]	validation-auc:0.72449
[1]	validation-auc:0.72983
[2]	validation-auc:0.73724
[3]	validation-auc:0.73755
[4]	validation-auc:0.73677
[5]	validation-auc:0.73724
[6]	validation-auc:0.73755
[7]	validation-auc:0.73681
[8]	validation-auc:0.73724
[9]	validation-auc:0.73694


[I 2024-04-30 03:04:42,828] Trial 0 finished with value: 0.82 and parameters: {'booster': 'dart', 'lambda': 3.152751444479729e-05, 'alpha': 0.44697926911065755, 'max_depth': 9, 'eta': 0.0003680399756490309, 'gamma': 1.59824440623675e-05, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 6.843493261342992e-06, 'skip_drop': 0.5219650333137934}. Best is trial 0 with value: 0.82.


[0]	validation-auc:0.68414
[1]	validation-auc:0.72503
[2]	validation-auc:0.72503
[3]	validation-auc:0.72503
[4]	validation-auc:0.72503
[5]	validation-auc:0.72503
[6]	validation-auc:0.72503
[7]	validation-auc:0.72503
[8]	validation-auc:0.72503
[9]	validation-auc:0.72503


[I 2024-04-30 03:04:42,919] Trial 1 finished with value: 0.82 and parameters: {'booster': 'dart', 'lambda': 0.005040833677881936, 'alpha': 0.01493042611697866, 'max_depth': 6, 'eta': 1.0982120182546449e-07, 'gamma': 0.004709906605828802, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 0.15744478199070983, 'skip_drop': 3.460852506624258e-08}. Best is trial 0 with value: 0.82.


...

[0]	validation-auc:0.72765
[1]	validation-auc:0.72765
[2]	validation-auc:0.76004
[3]	validation-auc:0.75849
[4]	validation-auc:0.76054
[5]	validation-auc:0.76994


[I 2024-04-30 03:04:49,977] Trial 98 pruned. Trial was pruned at iteration 5.


[0]	validation-auc:0.73782
[1]	validation-auc:0.73862
[2]	validation-auc:0.73862
[3]	validation-auc:0.73970
[4]	validation-auc:0.73916
[5]	validation-auc:0.73916


[I 2024-04-30 03:04:50,096] Trial 99 pruned. Trial was pruned at iteration 5.


FrozenTrial(number=0, state=TrialState.COMPLETE, values=[0.82], datetime_start=datetime.datetime(2024, 4, 30, 3, 4, 39, 593677), datetime_complete=datetime.datetime(2024, 4, 30, 3, 4, 42, 827969), params={'booster': 'dart', 'lambda': 3.152751444479729e-05, 'alpha': 0.44697926911065755, 'max_depth': 9, 'eta': 0.0003680399756490309, 'gamma': 1.59824440623675e-05, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 6.843493261342992e-06, 'skip_drop': 0.5219650333137934}, user_attrs={}, system_attrs={}, intermediate_values={0: 0.7244897959183674, 1: 0.7298267991407089, 2: 0.7372448979591837, 3: 0.737546992481203, 4: 0.7367749731471536, 5: 0.7372448979591837, 6: 0.737546992481203, 7: 0.7368085392051558, 8: 0.7372448979591837, 9: 0.7369428034371643}, distributions={'booster': CategoricalDistribution(choices=('gbtree', 'gblinear', 'dart')), 'lambda': FloatDistribution(high=1.0, log=True, low=1e-08, step=None), 'alpha': FloatDistribution(high=1.0, lo

In [ ]:
print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

Best trial:
  Value:  0.82
  Params: 
    booster: dart
    lambda: 3.152751444479729e-05
    alpha: 0.44697926911065755
    max_depth: 9
    eta: 0.0003680399756490309
    gamma: 1.59824440623675e-05
    grow_policy: depthwise
    sample_type: weighted
    normalize_type: forest
    rate_drop: 6.843493261342992e-06
    skip_drop: 0.5219650333137934


In [ ]:
start_time = time.time()

xgb_model = xgb.XGBClassifier(**trial.params)
xgb_model.fit(x_train,y_train)

end_time = time.time()
process_time = round(end_time-start_time,2)
print("Fitting XGBoost took {} seconds".format(process_time))

Fitting XGBoost took 24.25 seconds


In [ ]:
predictions = model.predict(x_test)

In [ ]:
print("Accuracy of model is {}%".format(accuracy_score(y_test,predictions) * 100))

Accuracy of model is 82.0%


In [ ]:
evaluate(y_true=y_test, y_pred=predictions)

Accuracy: 0.820
Accuracy for label 0: 0.855
Accuracy for label 1: 0.765

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.86      0.85       152
           1       0.77      0.77      0.77        98

    accuracy                           0.82       250
   macro avg       0.81      0.81      0.81       250
weighted avg       0.82      0.82      0.82       250


Confusion Matrix:
[[130  22]
 [ 23  75]]
